Built through **PDF Plumber** and depends on structure of the company's transcript releases - intuitive to adapt code. Consider the data in its raw form, consisting of text and \n (new lines). Normally, these are consistent for companies and therefore straightforward to implement rules to extract **dates, names,** etc. depending on where they are positioned in the format of the transcript.

For example, the dates for the below transcript form follow from 'Earnings Call' in the title. By finding the index of where that title ends: 

**date_idx = txt.find('EVENT DATE/TIME: ') + len('EVENT DATE/TIME: ')** 

We can find a range that encompasses the title and use split() to identify the month, day, year, etc.

In [ ]:
def transcript_convert(ticker):
    path = '...Transcripts/ALK'
    # path = os.path.join(*paths, ticker)
    pdf_list = [x for x in os.listdir(path) if x[-4:] == '.pdf']
    print(pdf_list)
    columns = ['particip', 'qna', 'transcript', 'date', 'company']
    data = pd.DataFrame(columns=columns)
    for f in pdf_list:
        found_start = False
        out = ''
        with pdfplumber.open(os.path.join(path, f)) as pdf:

            for i,v in enumerate(pdf.pages):
                curr_page = pdf.pages[i]
                txt = curr_page.extract_text()

                if i == 0:
                    if txt.find(ticker + ' -') != -1:
                        dt_idx = txt.find(ticker + ' -')
                        qt_txt = txt[dt_idx:].split('\n')
                        qt_txt = [x for x in qt_txt if x != ' '][0].strip(ticker + ' -')  # remove ' '
                    else:
                        qt_txt = [x for x in txt.split('\n') if 'Earnings' in x][0]
                    quarter = qt_txt.split()[0]
                    year = qt_txt.split()[1]
                    date_idx = txt.find('EVENT DATE/TIME: ') + len('EVENT DATE/TIME: ')
                    date_idx_end = date_idx + 50  # static character count for length of date
                    date = parse(txt[date_idx:date_idx_end].split('\n')[0]).strftime('%m-%d-%y')
                    comp_name = qt_txt.split(' Earnings Call')[0][qt_txt.split(' Earnings Call')[0].find(year)+ len(year) + 1:]

                elif i == 1:
                    corp_idx = txt.find('CORPORATE PARTICIPANTS')+len('CORPORATE PARTICIPANTS\n')
                    corp_idx_end = txt.find('\nCONFERENCE CALL PARTICIPANTS')
                    conf_idx = txt.find('CONFERENCE CALL PARTICIPANTS') + len('CONFERENCE CALL PARTICIPANTS\n')
                    conf_idx_end = txt.find('PRESENTATION')
                    corp_txt = txt[corp_idx:corp_idx_end].split('\n')
                    corp_name = [x.split(comp_name.split()[0])[0] for x in corp_txt]
                    corp_name = [x.strip() for x in corp_name if x.strip() != '']
                    corp_speaker1 = corp_name
                    corp_speaker2 = [x.split()[0] + ' ' + x.split()[1] for x in corp_txt if len(x.split()) >= 2]
                    corp_speaker3 = [x.split()[0] + ' ' + x.split()[2] for x in corp_txt if
                                     len(x.split()) >= 3]
                    corp_speaker4 = [x.split()[0] + ' ' + x.split()[1] + ' ' + x.split()[2] for x in corp_txt if
                                     len(x.split()) >= 3]

                    conf_txt = txt[conf_idx:conf_idx_end].split('\n')
                    conf_speaker1 = [x.split()[0] + ' ' + x.split()[1] for x in conf_txt if len(x.split()) >= 2]
                    conf_speaker2 = [x.split()[0] + ' ' + x.split()[2] for x in conf_txt if len(x.split()) >= 3]    # accounts for speakers with middle name
                    conf_speaker3 = [x.split()[0] + ' ' + x.split()[1] + ' ' + x.split()[2] for x in conf_txt if len(x.split()) >=3]    # accounts for speakers with middle name

                    speaker_list = corp_speaker1 + corp_speaker2 + corp_speaker3 + corp_speaker4 + conf_speaker1 + conf_speaker2 + conf_speaker3

                    columns = ['corp_particip','conf_particip']
                    df_speaker = pd.DataFrame(columns=columns)
                    corp_list = corp_speaker1 + corp_speaker2 + corp_speaker3 + corp_speaker4
                    conf_list = conf_speaker1 + conf_speaker2 + conf_speaker3
                    df_speaker['corp_particip'] = corp_list + (max(len(corp_list), len(conf_list)) -len(corp_list)) * ['']
                    df_speaker['conf_particip'] = conf_list + (max(len(corp_list), len(conf_list)) -len(conf_list)) * ['']
                    df_speaker.to_csv(os.path.join(path, ticker + '_' + date + '_speakers.csv'), index=False)

                # remove footer
                if i != 0 and i != len(pdf.pages)-1:
                    beginning_idx = txt.find('Earnings Call')+len('Earnings Call\n')
                    pg_num_idx = txt.find('\n' + str(i + 1))
                    txt = txt[beginning_idx:pg_num_idx]

                # remove disclaimer
                elif i ==len(pdf.pages)-1:  # if last page remove disclaimer
                    beginning_idx = txt.find('Earnings Call')+len('Earnings Call\n')
                    dis_idx = txt.find('DISCLAIMER')
                    txt = txt[beginning_idx:dis_idx]

                # write to file
                if found_start == True:
                    out = out+txt

                elif txt.find('PRESENTATION') != -1 and found_start == False:
                    found_start = True
                    start_idx = txt.find('PRESENTATION')+len('PRESENTATION\n')
                    out = out+txt[start_idx:]

        columns = ['particip', 'qna', 'transcript', 'date', 'company']
        df = pd.DataFrame(columns=columns)
        speaker = ''
        content = ''
        qna_start_j = 1000  # initialize
        j = 0
        for line in out.split('\n'):

            if line.find('QUESTIONS AND ANSWERS') != -1:
                qna_start_j = j + 2
            if len(line.split()) == 1:
                find_speaker = [x for x in ['Operator'] if (x in line)]
            else:
                find_speaker = [x for x in speaker_list if (x + ' -' in line)]  # + ' -'

            if find_speaker == []:
                content = content + ' ' + line
            else:
                if speaker != '':
                    if speaker in corp_list:
                        df.loc[j] = [speaker, 0, content, date, ticker]
                    else:
                        df.loc[j] = [speaker, 1, content, date, ticker]
                    content = ''
                    j += 1

                speaker = find_speaker[0]

        print(pdf.pages[0].extract_text())
        data = data.append(df)

    data.to_csv(os.path.join(path, ticker + '.csv'), index=False)
    print('converted: ' + os.path.join(path, ticker + '.csv'))

**Result is a file of multiple transcripts - by speaker, whether company personnel or external and date of the earnings call.**

In [ ]:
data = pd.read_csv('Transcripts_test.csv')
data['date'] = pd.Series(pd.to_datetime(data['date'], format='%m-%d-%y'))
data.info()

In [ ]:
def clean(text):
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t', '', str(text))
    # removing new line characters
    text = re.sub('\n ', '', str(text))
    text = re.sub('\n', ' ', str(text))
    # removing apostrophes
    text = re.sub("'s", '', str(text))
    # removing hyphens
    text = re.sub("-", ' ', str(text))
    text = re.sub("— ", '', str(text))
    # removing quotation marks
    text = re.sub('\"', '', str(text))
    # removing salutations
    text = re.sub("Mr\.", 'Mr', str(text))
    text = re.sub("Mrs\.", 'Mrs', str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))

    return text

In [ ]:
# preprocessing speeches
data['Transcript_clean'] = data['transcript'].apply(clean)

**Considering a subset of data - namely only company personnel.**

In [ ]:
data_subset = data[data['qna'] == 0]
data_subset = data_subset.drop(['particip'], axis=1)
data_subset = data_subset.groupby(['date'])['Transcript_clean'].apply(' '.join).reset_index()
print(data_subset.head(10))

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

**Breaking this down by whole sentences to analyse the contents of each.**

In [ ]:
def sentences(text):
    # split sentences and questions
    text = re.split('[.?]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

In [ ]:
# sentences
data['sent'] = data['Transcript_clean'].apply(sentences)

# create a dataframe containing sentences
df2 = pd.DataFrame(columns=['Sent', 'Year', 'Len'])

In [ ]:
row_list = []

for i in range(len(data)):
    for sent in data.loc[i, 'sent']:
        wordcount = len(sent.split())
        year = data.loc[i, 'Year']

        dict1 = {'Year': year, 'Sent': sent, 'Len': wordcount}
        row_list.append(dict1)

df2 = pd.DataFrame(row_list)
print(df2.head(10))

In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy import displacy
# import visualise_spacy_tree
from IPython.display import Image, display
# load english language model
nlp = spacy.load('en_core_web_sm', disable=['ner', 'textcat'])

**Information Extraction – Finding Keywords and using Dependencies**
SpaCy’s Matcher class. It allows us to match a sequence of words based on certain patterns and using regex to find sentences that only contain keywords

In [ ]:
def prog_sent(text):
    patterns = [r'\b(?i)' + 'coronavirus' + r'\b',
                r'\b(?i)' + 'Covid-19' + r'\b',
                r'\b(?i)' + 'lockdown' + r'\b',
                r'\b(?i)' + 'vaccine' + r'\b', ]

    output = []
    flag = 0
    for pat in patterns:
        if re.search(pat, text) != None:
            flag = 1
            break
    return flag

In [ ]:
def all_schemes(text, check):
    schemes = []

    doc = nlp(text)

    # initiatives
    prog_list = ['coronavirus', 'Covid-19',
                 'lockdown', 'vaccine']

    # pattern to match initiatives names
    pattern = [{'POS': 'DET'},
               {'POS': 'PROPN', 'DEP': 'compound'},
               {'POS': 'PROPN', 'DEP': 'compound'},
               {'POS': 'PROPN', 'OP': '?'},
               {'POS': 'PROPN', 'OP': '?'},
               {'POS': 'PR OPN', 'OP': '?'},
               {'LOWER': {'IN': prog_list}, 'OP': '+'}
               ]

    if check == 0:
        # return blank list
        return schemes

    # Matcher class object
    matcher = Matcher(nlp.vocab)
    matcher.add("matching", [pattern])
    matches = matcher(doc)

    for i in range(0, len(matches)):

        # match: id, start, end
        start, end = matches[i][1], matches[i][2]

        if doc[start].pos_ == 'DET':
            start = start + 1

        # matched string
        span = str(doc[start:end])

        if (len(schemes) != 0) and (schemes[-1] in span):
            schemes[-1] = span
        else:
            schemes.append(span)

    return schemes

In [ ]:
def sent_subtree(text):
    # pattern match for schemes or initiatives
    patterns = [r'\b(?i)' + 'coronavirus' + r'\b',
                r'\b(?i)' + 'Covid-19' + r'\b',
                r'\b(?i)' + 'lockdown' + r'\b',
                r'\b(?i)' + 'vaccine' + r'\b']

    schemes = []
    doc = nlp(text)
    flag = 0
    # if no initiative present in sentence
    for pat in patterns:

        if re.search(pat, text) != None:
            flag = 1
            break

    if flag == 0:
        return schemes

    # iterating over sentence tokens
    for token in doc:

        for pat in patterns:

            # if we get a pattern match
            if re.search(pat, token.text) != None:

                word = ''
                # iterating over token subtree
                for node in token.subtree:
                    # only extract the proper nouns
                    if (node.pos_ == 'PROPN'):
                        word += node.text + ' '

                if len(word) != 0:
                    schemes.append(word)

    return schemes